# EPA Air Pollution Research - Impact of COVID-19 on air quality improvement in EJ Communities

## Background (known, known):
* high correlation between low EJ communities and low air quality

## Goal (known, unknown):
* h0: COVID-19 improved air quality equally in all EJ communities (same % improvement)
* h1: COVID-19 improved air quality more/less in low EJ communities (different % improvement)
* control: exclude daily measurement of areas effect by wild fire (identify date, area)

## Procedure:
1. analyze Ozone, PM2.5, demographic data
2. remove data with wild fire effect

# 1.0 Import Tools

Let's import libraries required for this analysis.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import constants, stats
from datetime import datetime as dt

!pip install researchpy
!pip install openpyxl
!pip install mapbox

import researchpy as rp

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import plotly
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

from mapbox import Geocoder
geocoder = Geocoder(access_token="pk.eyJ1IjoiamFlbGluMjE1IiwiYSI6ImNsMTgyOWpjNjA5N2IzaWpyZHZoY3hqNXgifQ.mGUBuIT7Il5ZRLKJviaWyw")

# 2.0 Preprocess Data

Let's load daily average air quality dataset (2019, 2020) from EPA AirNow. For easy manipulation, we will remove any trailing empty spaces from column names and make it all lower case. Let's see how many data point we have from each 2019, 2020.

In [2]:
# import data
epa_2019 = pd.read_excel('../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2019.xlsx', sheet_name='Sheet1', parse_dates=['DATE'])
epa_2020 = pd.read_excel('../input/phase-ii-widsdatathon2022/epa/epa/Datathon_EPA_Air_Quality_Demographics_Meteorology_2020.xlsx', sheet_name='Sheet1', parse_dates=['DATE'])
#life_expectancy = pd.read_csv('../input/usaleep-life-expectancy/US_B.CSV')

epa_2019 = epa_2019.rename(columns=str.strip)
epa_2019 = epa_2019.rename(columns=str.lower)
epa_2020 = epa_2020.rename(columns=str.strip)
epa_2020 = epa_2020.rename(columns=str.lower)
print('2019 (rows, columns): ', epa_2019.shape)
print('2020 (rows, columns): ', epa_2020.shape)
print('Total daily data points (2019-2022):', epa_2019.shape[0] + epa_2020.shape[0])

In [3]:
epa_2019

In [4]:
# merge 2019, 2020 
epa = epa_2019.append(epa_2020)

# rename columns 
old_cols = epa_2019.columns.values.tolist()
new_cols = ['sensor_id','sensor_lat','sensor_long','county','state','cbsa','color','income','language','education','date','temp','humidity','wind_speed','wind_direction','pm25','ozone','no2','co','so2','lead','benzene']
for i in range(len(old_cols)):
    epa = epa.rename(columns={old_cols[i]:new_cols[i]})

# print missing values
print('Missing values (2019-2020): \n')
epa.isnull().sum().sort_values(ascending=False)

In [5]:
epa[epa.education.isnull()]

**No missing data (id, geo-location, important pollutants (Ozone, PM25)):**
* date                   
* sensor_lat                    
* pm25                   
* ozone                  
* state                  
* county                 
* sensor_long                   
* sensor_id                     

**Missing data (a little = demographic data):**
* education             
* language              
* income                
* color                 

**Missing data (a lot = other pollutants, weather, cbsa data):**
* lead              
* benzene         
* co                
* so2               
* humidity          
* wind_speed        
* wind_direction    
* no2               
* temp              
* **cbsa**

`Core-Based Statistical Area (CBSA)` is officially specified by the Office of Management and Budget, the CBSA is the Census summary level representing a “metropolitan area.” Technically, CBSAs are either metropolitan or micropolitan statistical areas—both describe a specific group of counties (or sometimes just one county) around an urban core.

While metro areas vary greatly in size, for some kinds of analysis they are better units of comparison than the single Census place at the core.

In [6]:
epa[['cbsa0', 'cbsa1']] = epa['cbsa'].str.split(',', 1, expand=True)
# df = epa.filter(regex='cbsa?')
# print(df.nunique())

# df = df.reindex(sorted(df.columns), axis=1)
# df.head(2)
epa.head(2)

In [7]:
us_states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_states.items()))

In [8]:
epa.head(2)

In [9]:
epa['cbsa'].isnull().sum()

In [10]:
# replace NaN to 0
epa['cbsa0'] = epa['cbsa0'].replace(np.nan, 0)
epa['cbsa1'] = epa['cbsa1'].replace(np.nan, 0)
epa['cbsa'] = epa['cbsa'].replace(np.nan, 0)
epa['cbsa0'].isnull().sum()

In [11]:
epa.shape

In [12]:
epa.head(2)

In [13]:
epa = epa.reset_index()
del epa['index']
epa

In [14]:
# find index of rows with missing cbsa 
ind = epa.loc[epa.cbsa == 0].index.tolist()

In [15]:
# replace 
old = epa.iloc[ind]['cbsa0'].tolist()
new_cbsa0 = epa.iloc[ind]['county'].tolist()
new_cbsa1 = epa.iloc[ind]['state'].map(us_states).tolist()
#new_cbsa = new_cbsa0.concat(', ', new_cbsa1)

epa.at[ind, 'cbsa0'] = new_cbsa0
# epa.ix['cbsa1', ind] = new_cbsa1
#epa.iloc[ind]['cbsa'] = new_cbsa0 + new_cbsa1
#epa
#epa['cbsa0']
#epa.iloc['cbsa0']


### **`replace 0 values`** with concatenation of other column values

In [16]:
# replace 0 with other values from 'county' and 'state' columns
#df.loc[df[<some_column_name>] == <condition>, [<another_column_name>]] = <value_to_add>

epa.loc[epa.cbsa == 0, ['cbsa0']] = epa.loc[epa.cbsa == 0]['county']
epa.loc[epa.cbsa == 0, ['cbsa1']] = epa['state'].map(us_states)
epa.loc[epa.cbsa == 0, ['cbsa']] = epa.loc[epa.cbsa == 0]['cbsa0'] + ', ' + epa.loc[epa.cbsa == 0]['cbsa1']

# check the updated values
epa[epa.county == epa.cbsa0]

-----------------
## Export to CSV (EPA preprocessed - CBSA filled)
-----------------

In [17]:
epa.to_csv('epa_preprocessed.csv')

## Check Number of Sensors, Unique Dates per each EJ Community (CBSA)

In [18]:
print('Counts of Sensors, Daily data points: ')
counts_df = epa.groupby(by='cbsa').agg('nunique')
summary_counts = counts_df[['sensor_id', 'date']].sort_values('sensor_id', ascending=False)
summary_counts
#summary_counts.describe()
#summary_counts.shape

**Summary counts:**

* Unique CBSA Communities: 306
* Unique Sensors: 1-11 (median = 1.0)
* Unique Dates: 41-729 (median = 640)

## Date -> Year, Month, Date:

In [19]:
epa['year'] = pd.DatetimeIndex(pd.to_datetime(epa['date'])).year
epa['month'] = pd.DatetimeIndex(pd.to_datetime(epa['date'])).month
epa['day'] = pd.DatetimeIndex(pd.to_datetime(epa['date'])).day
epa['weekday'] = epa['date'].dt.dayofweek
epa.head(2)

# 3.0 Analyze Data (Important pollutants + Demographic)

## 3.1 Primary Pollutants
According to numerous studies, ozone and fine particulates (PM25) have the most siginificant health impact to humans. Therefore, let's identify if the concentration of these pollutants have been increasing/decreasing due to COVID-19.

In [20]:
epa_primary_pollutants = epa[['cbsa', 'cbsa0','cbsa1','date','year','month','day','weekday','ozone','pm25']]
epa_primary_pollutants = epa_primary_pollutants.set_index('date')

------------------
## Export to CSV (preprocessed EPA Primary Pollutants)
------------------

In [21]:
epa_primary_pollutants.to_csv('primary_pollutants.csv')
epa_primary_pollutants

In [22]:
data_2019 = pd.DataFrame(epa_primary_pollutants['2019-01-01':'2019-12-31'])
data_2020 = pd.DataFrame(epa_primary_pollutants['2019-01-01':'2019-12-31'])
before_covid = pd.DataFrame(epa_primary_pollutants['2019-01-01':'2020-03-31'])
during_covid = pd.DataFrame(epa_primary_pollutants['2020-04-01':'2019-12-31'])

data_all = epa_primary_pollutants

## 3.1.1 US-wide Primary Pollutants (2019-2020)

Some spikes are observed around September 2020. Let's investigate **wild fire** events around this time.

In [23]:
fig, ax = plt.subplots(1,1, figsize=(10,7))
ax.set(title='US-wide Ozone, PM2.5 Concentration Change (2019 vs 2020)')

g1 = sns.scatterplot(data=data_all, x=data_all.index, y='ozone', label='Ozone', hue='year')
ax2 = plt.twinx()
g2 = sns.scatterplot(data=data_all, x=data_all.index, y='pm25', label='PM 2.5', ax=ax2, color='green')
sns.move_legend(g1, "upper right", bbox_to_anchor=(1., 1))
sns.move_legend(g2, "upper right", bbox_to_anchor=(1., 0.8))
fig.autofmt_xdate()
plt.show()

## 3.1.2 New York Pollutants (2019-2020)

In [24]:
data_ny = epa_primary_pollutants[epa_primary_pollutants.cbsa.str.contains('New York')]

fig, ax = plt.subplots(1,1, figsize=(10,7))
g1 = sns.scatterplot(x=data_ny.index, y='ozone', data=data_ny, hue='year', label='Ozone')
ax2 = plt.twinx()
g2 = sns.scatterplot(x=data_ny.index, y='pm25', data=data_ny, color='green', ax=ax2, label='PM 2.5')
sns.move_legend(g1, "upper right", bbox_to_anchor=(1., 1))
sns.move_legend(g2, "upper right", bbox_to_anchor=(1., 0.8))
fig.autofmt_xdate()
plt.show()

# 3.2 Wile Fire Impact

## 3.2.1 California Wild Fire (2020)

In 2020, California wiledfires started Feb 15 and lasted until Dec 31. Let's filter the rows by 'cbsa1 = CA'and see any spikes are observed in pollutant concentration. This will inform us the signature of wildfile in our dataset. Then, we can look for similar trends in other areas. COVID-19 lockdown impact can only be analyzed properly when we isolate wild fire impact. Let's start by looking at California region.

In [25]:
# california data
ca = epa_primary_pollutants[epa_primary_pollutants.cbsa1 == 'CA']
ca

In [26]:
# visualize california data
data_ca = ca

fig, ax = plt.subplots(1,1, figsize=(10,7))
g1 = sns.scatterplot(x=data_ca.index, y='ozone', data=data_ca, hue='year', label='Ozone')
ax2 = plt.twinx()
g2 = sns.scatterplot(x=data_ca.index, y='pm25', data=data_ca, color='green', ax=ax2, label='PM 2.5')
sns.move_legend(g1, "upper right", bbox_to_anchor=(1., 1))
sns.move_legend(g2, "upper right", bbox_to_anchor=(1., 0.8))
fig.autofmt_xdate()
plt.show()

Based on the spikes of PM 2.5 concentration, we can suspect California Wild Fire likely happend around these dates:

* 2019-08-20 (spike)
* 2020-01-01 (spike)
* 2020-08-17 (lasting started)
* 2020-09-15 (lasting peak)
* 2020-10-10 (lasting ended, after 2 months)


# 3.3. COVID-19 Impact

**2020**
* Mar 19: California lock down
* Mar/Apr: World lock down
* May: re-opening


# 4.0 Geolocation Mapping

In [27]:
df = epa

geometry = [Point(xy) for xy in zip(df['sensor_long'], df['sensor_lat'])]
gdf = GeoDataFrame(df, geometry=geometry)   
gdf.head(2)

In [28]:
#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(30, 18)), marker='o', color='red', markersize=15);

In [29]:
fig, ax = plt.subplots(1,1)
ax = sns.scatterplot(data=epa, x='date',y='no2')
#ax.get_legend().remove()
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [30]:
epa.head(2)

# 5.0 Missing Data (per EJ Community)

## 5.1 Missing Ozone data

In [31]:
fig, ax = plt.subplots(1,1, figsize=(10,40))
ax = sns.scatterplot(x='date', y='cbsa', data = epa, size='ozone',markers='.')
ax.set(title='Ozone')
ax.get_legend().remove()
ax.set(autoscaley_on=True,ymargin=0.01)
fig.autofmt_xdate()
plt.tight_layout()

## 5.2 Missing PM 2.5 data

In [32]:
fig, ax = plt.subplots(1,1, figsize=(10,40))
ax = sns.scatterplot(x='date', y='cbsa', data = epa, size='pm25',markers='.')
ax.set(title='PM 2.5')
ax.get_legend().remove()
ax.set(autoscaley_on=True,ymargin=0.01)
fig.autofmt_xdate()
plt.tight_layout()

## 5.3 Missing NO2 data

In [33]:
fig, ax = plt.subplots(1,1, figsize=(10,40))
ax = sns.scatterplot(x='date', y='cbsa', data = epa, size='no2',markers='.')
ax.set(title='NO2')
ax.get_legend().remove()
ax.set(autoscaley_on=True,ymargin=0.01)
fig.autofmt_xdate()
plt.tight_layout()

# 6.0 Demographic & Air Pollution (COVID-19 impact)

In [34]:
color_co = epa.groupby('cbsa').agg({'color': ['mean', 'count'], 'co': 'mean'}).sort_values([('co', 'mean')], ascending=False)
color_co_clean = color_co.dropna()
color_co_clean

# 7.0 Correlation

In [35]:
corr = epa.corr()

In [48]:
mask = np.zeros_like(corr)
fig, ax = plt.subplots(figsize=(14,8)) 
sns.heatmap(corr, mask=mask, cmap="YlGnBu", ax=ax, annot=True, fmt='.1f')
plt.show()

Strong **POSITIVE CORRELATION** between PEOPLE_OF_COLOR_FRACTION, LOW_INCOME_FRACTION, LINGUISTICALLY_ISOLATED_FRACTION, LESS_THAN_HS_EDUCATION_FRACTION
Strong **POSITIVE CORRELATION** between NO2_PPM, CO_PPM, BENZENE_PPMBC,

In [49]:
sns.lmplot(data=corr, x='color', y='co')